In [ ]:
# import for website redirection
from IPython.display import display, Javascript
# imports for the main menu
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

class WindowManager:
    def __init__(self):
        self.current_window = 1
        self.selected_test = ""
        self.panels = Panels(self)
        self.consent_given = False
        self.window_dictionary = {
            1: self.panels.home_screen,
            2: self.panels.consent_panel,
            3: self.panels.form_panel,
            4: self.panels.additional_info_panel,
            5: self.panels.thank_you_panel
        }
        self.output = widgets.Output()
        display(self.output)
        self.show_panel()
        
    def advance_panel(self):
        self.current_window += 1
        self.show_panel()

    def show_panel(self):
        with self.output:
            clear_output()
            panel_function = self.window_dictionary.get(self.current_window)
            panel_function()
            
    def link_redirection(self):
        display(Javascript(f"""window.location.href = 'http://localhost:8866/voila/render/{self.selected_test}/{self.selected_test}.ipynb';"""))
                
class Panels:
    def __init__(self, window_manager):
        self.window_manager = window_manager
        self.output = widgets.Output()

    def home_screen(self):
        home_text = widgets.HTML(
            value="<div style='text-align: center; font-size: 16px;'>"
                  "<h2>Welcome to the UCL BIOS0030 cognitive tasks main menu</h2>"
                  "<p>You will have a chance to complete four different cognitive tasks, each testing different abilities.<br>"
                  "You may wish to allocate 10-15 minutes to complete all of them."
                  "<br>Most importantly, <u>take them with a grain of salt</u>. "
                  "<b>Have fun!</b></div>",
            layout=widgets.Layout(justify_content='center')
        )
        display(home_text)

        choices_dictionary = {
            "ANS_Test": {
                "photo": widgets.Image(value=open("ans.png", "rb").read(), format='png', width=120, height=120),
                "button": widgets.Button(description="ANS Test")
            },
            "Maths_Test": {
                "photo": widgets.Image(value=open("math_photo.png", "rb").read(), format='png', width=120, height=120),
                "button": widgets.Button(description="Maths Test")
            },
            "Memory_Test": {
                "photo": widgets.Image(value=open("memory.png", "rb").read(), format='png', width=120, height=120),
                "button": widgets.Button(description="Memory Test")
            },
            "Spatial_Test": {
                "photo": widgets.Image(value=open("puzzle.png", "rb").read(), format='png', width=120, height=120),
                "button": widgets.Button(description="Spatial Test")
            }
        }

        def click_consequence(test):
            self.window_manager.selected_test = test
            def button_action(btn):
                if not self.window_manager.consent_given: 
                    self.window_manager.advance_panel()
                else:
                    self.window_manager.link_redirection()
            return button_action

        for test, widget in choices_dictionary.items():
            widget["button"].on_click(click_consequence(test))

        # Grid layout
        grid = widgets.GridspecLayout(2, 4)
        for i, (test, widget) in enumerate(choices_dictionary.items()):
            grid[0, i] = widget["photo"]
            grid[1, i] = widget["button"]

        vboxed_grid = widgets.VBox([grid],
                                   layout=widgets.Layout(align_items='center', margin='40px 0 0 0'))
        display(vboxed_grid)

    def consent_panel(self):
        consent_description = widgets.HTML(
            value="<div style='text-align: center; font-size: 16px;'>"
            "<b>Please read:</b> First, we wish to record your response data to an anonymised public data repository.<br>"
            "Your data will be used for educational teaching purposes practicing data analysis and visualisation.<br>"
            "The conscent you give here will be valid throught your session on our website.<br>"
            "<br>Do you conscent to having your data saved and uploaded?</div>",
            layout=widgets.Layout(justify_content='center')
        )

        yes_button = widgets.Button(description="Yes", layout=widgets.Layout(width='100px'))
        no_button = widgets.Button(description="No", layout=widgets.Layout(width='100px'))

        def yes_button_click(btn):
            self.window_manager.consent_given = True
            self.window_manager.advance_panel()

        def no_button_click(btn):
            self.window_manager.current_window = 0
            self.window_manager.advance_panel()

        yes_button.on_click(yes_button_click)
        no_button.on_click(no_button_click)

        button_box = widgets.HBox([yes_button, no_button],
                                  layout=widgets.Layout(justify_content='center', align_items='center'))

        panel = widgets.VBox([consent_description, button_box],
                             layout=widgets.Layout(display='flex', flex_flow='column', align_items='center', width='100%'))
        display(panel)

    def form_panel(self):

        instructions_html = widgets.HTML(
                value="<div style='text-align: center; font-size: 16px;'>" 
                      "<b>We need to get a better idea about who you are.</b><br>"
                      "Kindly please fill out the form below.</div>",
                layout=widgets.Layout(justify_content='center')
            )

        anonymised_id_explanation = widgets.HTML(
            value="<div style='text-align: center; margin-top: 20px; margin-bottom: 10px;'>"
                  "<b>To generate ananymysed ID unique user identifier</b> please enter:<br>"
                  "- two letters based on the initials (first and last name) of a childhood friend<br>"
                  "- two letters based on the initials (first and last name) of a favourite actor/actress<br>"
                  "<i>Example: If your friend was called Charlie Brown and film star was Tom Cruise, then your unique identifier would be CBTC</i></div>"
        )


        # Form fields
        id_widget = widgets.Text(description="ID:", layout=widgets.Layout(width='250px'))
        age_widget = widgets.IntText(description="Age:", layout=widgets.Layout(width='250px'))
        gender_widget = widgets.Dropdown(description="Gender:", options=['Male', 'Female', 'Other'], layout=widgets.Layout(width='250px'))
        year_widget = widgets.Dropdown(description="Course Year:", options=['1', '2', '3', '4', 'not applicable'], layout=widgets.Layout(width='250px'))
        course_widget = widgets.Text(description="Course:", layout=widgets.Layout(width='250px'))

        # VBox for form fields with left margin
        form_fields_vbox = widgets.VBox([id_widget, age_widget, gender_widget, year_widget, course_widget],
                                        layout=widgets.Layout(justify_content='space-around', margin='0 0 0 38%'))

        # Submit button centered
        submit_button = widgets.Button(description="Submit")
        def click_consequence(btn):
            self.window_manager.advance_panel()
        submit_button.on_click(click_consequence)

        submit_button_vbox = widgets.VBox([submit_button], layout=widgets.Layout(align_items='center'))

        display(instructions_html)
        display(anonymised_id_explanation)
        display(form_fields_vbox)
        display(submit_button_vbox)

    def additional_info_panel(self):

        instructions_html = widgets.HTML(
                value="<div style='text-align: center; font-size: 16px;'>" 
                      "Lastly, please aswer a few additional questions.<br>"
                      "As before, if you do not wish to share any of your data and remain anonymous, simply leave the fields empty before clicking.<br>"
                      "<br><b>How are you doing?</b></div>",
                      
                layout=widgets.Layout(justify_content='center')
            )

        caffeine_widget = widgets.Dropdown(description="Caffeine:", options=['8h ago', '12h ago', '24h ago', '2+ days', 'None'])
        sleep_widget = widgets.Dropdown(description="Sleep:", options=['< 4h', '5h', '6h', '7h', '8h+'])
        exercise_widget = widgets.Dropdown(description="Exercise:", options=['Daily', 'Several/week', 'Weekly', 'Rarely', 'Never'])
        stress_level_widget = widgets.Dropdown(description="Stress:", options=['Very Low', 'Low', 'Moderate', 'High', 'Very High'])
        submit_button = widgets.Button(description="Submit")

        def click_consequence(btn):
            self.window_manager.advance_panel()

        submit_button.on_click(click_consequence)

        info_widgets = widgets.VBox([caffeine_widget, sleep_widget, exercise_widget, stress_level_widget],
                                   layout=widgets.Layout(justify_content='space-around', margin='0 0 0 36%'))

        submit_button_vbox = widgets.VBox([submit_button], layout=widgets.Layout(align_items='center'))

        display (instructions_html)
        display(info_widgets)
        display(submit_button_vbox)


    def thank_you_panel(self):
        
        thank_you_message = widgets.Label("Thank you! Next test will be shown in a moment.")
        thank_you_message = widgets.HTML(
                value="<div style='text-align: center; font-size: 16px;'>" 
                      "<b>Thank you! Next test will be shown in a moment.</div>",
                layout=widgets.Layout(justify_content='center')
            )
        display(thank_you_message)

        time.sleep(5)
        self.window_manager.consent_given = True
        self.window_manager.link_redirection()

# Instantiate and run the WindowManager
run = WindowManager()

Output()